In [10]:
# Workspace problem with several narrow gaps

import torch
import torch.nn as nn

from functools import partial
import numpy as np
from numpy.random import default_rng
from io import StringIO
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import matplotlib.animation as animation
import matplotlib.patches as patches
import sys
import time
import cv2
from celluloid import Camera
import time
from IPython.display import HTML

sys.path.append('/home/oscar_palfelt/MSc_thesis/ompl/py-bindings')
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og

rng = default_rng(900)

cuda = False
DEVICE = torch.device("cuda" if cuda else "cpu")

In [14]:
# neural network parameters
mb_size = 256 # mini batch dim
h_Q_dim = 512*4 # encoder dim
h_P_dim = 512*4 # decoder dim

# problem dimenc_dimsions
nDrawnSamples = 18 # number of dependent samples to draw during smapling (length of predicted control series)
nPreviousStates = 1 # number of previous states to condition on the steering prediction
dim = 4 # (x, y, yaw, steer)
dimObs = 3*2
dataElements = (nDrawnSamples + nPreviousStates) * dim + dim - 1 + dimObs # steer sample, current pose (3D: x,y,yaw), goal (3D), dimObs (2D x 3)

z_dim = 1 # latent dim
X_dim = nDrawnSamples*dim # steering
c_dim = dataElements - X_dim # dimension of conditioning variable

In [15]:
# define pytorch networks
# based on https://github.com/Jackson-Kang/Pytorch-VAE-tutorial/blob/master/.ipynb_checkpoints/01_Variational_AutoEncoder-checkpoint.ipynb

class Encoder(nn.Module):
    
    def __init__(self, input_dim=X_dim+c_dim, hidden_dim=h_Q_dim, latent_dim=z_dim):
        super(Encoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        self.z_mu = nn.Linear(hidden_dim, latent_dim)
        self.z_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x):

        seq = self.network(x)

        return self.z_mu(seq), self.z_logvar(seq)


class Decoder(nn.Module):
    def __init__(self, latent_dim=z_dim+c_dim, hidden_dim=h_P_dim, output_dim=X_dim):
        super(Decoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        
        return self.network(x)


class NeuralNetwork(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(NeuralNetwork, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
    
    def reparameterization(self, mean, logvar):
        epsilon = torch.randn_like(mean).to(DEVICE)        # sampling epsilon        
        z = mean + torch.exp(0.5 * logvar) * epsilon       # reparameterization trick
        return z

    def forward(self, x, c, encode=True):
        if encode:
            z_mu, z_logvar = self.Encoder(torch.cat((x, c), dim=1))
            z = self.reparameterization(z_mu, z_logvar)

            y = self.Decoder(torch.cat((z, c), dim=1))
            
            return y, z_mu, z_logvar
        else:
            z = x
            y = self.Decoder(torch.cat((z, c), dim=1))    

            return y

network = torch.load('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/networks/maze_26.pt', map_location=torch.device(DEVICE))

In [16]:
# read in data from .txt file, re-arrange to allow drawing multiple depedent samples

filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/motion_planning/data/ctrlDataPfollowMaze_70kplans'
rawdata = np.genfromtxt(filename, delimiter=',', dtype='d', usecols=[0,1,2,3,7,8,9,10,11,12,13,14,15]) # disregard init

_, pathsIdx = np.unique(rawdata[:,4:], axis=0, return_index=True)
pathsIdx.sort()

pathsLengths = np.roll(pathsIdx, -1) - pathsIdx
pathsLengths[-1] = rawdata.shape[0] - np.sum(pathsLengths[:-1])

validLengthsIdx = np.argwhere(pathsLengths >= nPreviousStates + nDrawnSamples*1.6) 
validPlansIdx = pathsIdx[validLengthsIdx]

data = np.zeros(shape=(1, dataElements)) # steering angles, samples (x,y,yaw), goal(x,y,yaw)
tempdata = np.copy(data) # steering angles, samples (x,y,yaw), goal(x,y,yaw)
for ci, i in enumerate(validPlansIdx[:-1].reshape(-1)):
    #rawdata[i:i+pathsLenghts[ci], dim-1] = uniform_filter1d(rawdata[i:i+pathsLenghts[ci], dim-1], size=1, origin=0, mode='nearest', axis=0) # average over steering
    for j in range(pathsLengths[ci] - (nPreviousStates + nDrawnSamples - 1)):
        sample = np.arange(start=j, stop=j + nPreviousStates + nDrawnSamples)

        if np.any(np.abs(np.diff(rawdata[i + sample, 0])) > 0.05) or np.any(np.abs(np.diff(rawdata[i + sample, 1])) > 0.05):
            break # ignore duplicate plans (data generation is not perfect)

        tempdata = np.vstack((tempdata, np.append(rawdata[i + sample, :dim].reshape(1, (nPreviousStates + nDrawnSamples)*dim), rawdata[i, dim:].reshape(1, dim-1+dimObs), axis=1))) 
        if ci % 500 == 0:
            data = np.vstack((data, tempdata[1:]))
            tempdata = np.zeros(shape=(1, dataElements))

rng.shuffle(data) # shuffle data

pathData = np.copy(data[:, :-dimObs])
obsData = np.copy(data[:, -dimObs:])

numEntries = data.shape[0]
print(data.shape)

(189567, 85)


In [6]:
# change conditions to occupancy grid
def isSampleFree(sample, obs):
    for o in list(range(0,obs.shape[0]//(2*dimW))): # python 2 -> 3: use list(), use //
        isFree = 0
        for d in range(0,sample.shape[0]):
            if (sample[d] < obs[2*dimW*o + d] or sample[d] > obs[2*dimW*o + d + dimW]):
                isFree = 1
                break
        if isFree == 0:
            return 0
    return 1

gridSize = 11
dimW = 3
plotOn = False;

# process data into occupancy grid
conditions = obsData
conditionsOcc = np.zeros([numEntries,gridSize*gridSize])
occGridSamples = np.zeros([gridSize*gridSize, 2])
gridPointsRange = np.linspace(0,1,num=gridSize)

idx = 0;
for i in gridPointsRange:
    for j in gridPointsRange:
        occGridSamples[idx,0] = i
        occGridSamples[idx,1] = j
        idx += 1;

start = time.time();
for j in range(0,numEntries,1):
    dw = 0.1
    gap1 = conditions[j,0:2]
    gap2 = conditions[j,2:4]
    gap3 = conditions[j,4:6]

    obs1 = [0, gap1[1]-dw, -0.5,             gap1[0], gap1[1], 1.5]
    obs2 = [gap2[0]-dw, 0, -0.5,             gap2[0], gap2[1], 1.5];
    obs3 = [gap2[0]-dw, gap2[1]+dw, -0.5,    gap2[0], 1, 1.5];
    obs4 = [gap1[0]+dw, gap1[1]-dw, -0.5,    gap3[0], gap1[1], 1.5];
    obs5 = [gap3[0]+dw, gap1[1]-dw, -0.5,    1, gap1[1], 1.5];
    obs = np.concatenate((obs1, obs2, obs3, obs4, obs5), axis=0)
    
    if j % 5000 == 0:
        print('Iter: {}'.format(j))
        
    occGrid = np.zeros(gridSize*gridSize)
    for i in range(0,gridSize*gridSize):
        occGrid[i] = isSampleFree(occGridSamples[i,:],obs)
    conditionsOcc[j,:] = occGrid
    
end = time.time();
print('Time: ', end-start)

Iter: 0
Iter: 5000
Time:  2.1282169818878174


In [7]:
ratioTestTrain = 0.8;
numTrain = int(numEntries*ratioTestTrain)

nextStates = pathData[:, nPreviousStates*dim:-dim+1]

prevStatesConditions = pathData[:, :nPreviousStates * dim]
goalStateCondition = pathData[:, -dim+1:]

cs = np.concatenate((prevStatesConditions, goalStateCondition, conditionsOcc), axis=1)

c_dim = cs.shape[1]
c_train = cs[0:numTrain,:]  
c_test = cs[numTrain:numEntries,:]

print(cs.shape)

(5880, 128)


In [9]:
# define planning problem

def getThresholdPathLengthObj(si):
     obj = ob.PathLengthOptimizationObjective(si)
     obj.setCostThreshold(ob.Cost(4.0))
     return obj


def isStateValid(spaceInformation, state):
    # perform collision checking or check if other constraints are
    # satisfied
    u = int(np.floor(state.getX() * occGrid.shape[1])) # right pointing image axis
    v = int(np.floor(occGrid.shape[0] * (1 - state.getY()))) # down pointing image axis

    if spaceInformation.satisfiesBounds(state):
        return occGrid[v,u] > 0


def propagate(start, control, duration, state):
    global nExploredStates
    nExploredStates += 1


    state.setX(start.getX() + v * duration * np.cos(start.getYaw()))
    state.setY(start.getY() + v * duration * np.sin(start.getYaw()))
    state.setYaw(start.getYaw() + control[0] * duration)


def problemDef():
    # construct the state space we are planning in
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(0.001)
    bounds.setHigh(0.999)
    space.setBounds(bounds)

    # create a control space
    cspace = oc.RealVectorControlSpace(space, 1)

    # set the bounds for the control space
    cbounds = ob.RealVectorBounds(1)
    cbounds.setLow(-1.5)
    cbounds.setHigh(1.5)
    cspace.setBounds(cbounds)

    ss = oc.SimpleSetup(cspace)
    ss.setStatePropagator(oc.StatePropagatorFn(propagate))
    si = ss.getSpaceInformation()
    si.setPropagationStepSize(.07)
    si.setMinMaxControlDuration(1, 10)
    planner = oc.RRT(si)


    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isStateValid, ss.getSpaceInformation())))
    ss.setPlanner(planner)
    ss.getProblemDefinition().setOptimizationObjective(getThresholdPathLengthObj(si))
    
    return ss


def plan(planObj, initState, goalState):

    space = ob.SE2StateSpace()
    
    start = ob.State(space)
    start().setX(initState[0])
    start().setY(initState[1])
    start().setYaw(initState[2])

    goal = ob.State(space)
    goal().setX(goalState[0])
    goal().setY(goalState[1])
    goal().setYaw(goalState[2])
    
    planObj.setStartAndGoalStates(start, goal, 0.1)

    timeTerminationCondition = ob.timedPlannerTerminationCondition(20.)
        
    exactSolTerminationCondition = ob.exactSolnPlannerTerminationCondition(planObj.getProblemDefinition())
    costTerminationCondition = ob.CostConvergenceTerminationCondition(planObj.getProblemDefinition(), epsilon=0.1)

    solved = planObj.solve(ob.plannerOrTerminationCondition(costTerminationCondition, timeTerminationCondition))


In [11]:
class VehicleState():
    def __init__(self):
        self.x = 0
        self.y = 0
        self.yaw = 0
        self.v = 0

In [12]:
# plot the latent space
num_viz = 800 # number of samples to draw in latent space

dt = 0.4 # controller time step
L = 0.2 # vehicle length constant
v =  0.1 # vehicle speed

pdef = problemDef()
t = np.tile(np.arange(nDrawnSamples), reps=(num_viz, 1)) * dt

nTests = 1000
nCompletedTests = 0
nAttemptedTests = 0
animate = False
CVAEtime = np.zeros(shape=nTests)
RRTtime = np.zeros(shape=nTests)
nCVAEstates = np.zeros(shape=nTests)
nRRTstates = np.zeros(shape=nTests)
nCVAEsuccess = 0
nRRTsuccess = 0
nCVAEfails = 0
nRRTfails = 0
while nCompletedTests < nTests:
    nAttemptedTests += 1

    collision = False

    CVAEstartTime = time.time()
    vizIdx = np.random.randint(numTrain,numEntries-1) # chose a random test scenario # np.random.choice(uniqueMapInd)

    occGrid = conditionsOcc[vizIdx].reshape(gridSize, gridSize)
    c_sample_seed = cs[vizIdx,:]
    c_sample = torch.from_numpy(np.repeat([c_sample_seed],num_viz,axis=0)).float().to(DEVICE)

    xPrev = prevStatesConditions[vizIdx, ::dim]
    yPrev = prevStatesConditions[vizIdx, 1::dim]
    yawPrev = prevStatesConditions[vizIdx, 2::dim]
    steerPrev = prevStatesConditions[vizIdx, 3::dim]
    steerNext = nextStates[vizIdx, 3::dim]

    truckState = VehicleState()
    truckState.x = xPrev[-1]; truckState.y = yPrev[-1]; truckState.yaw = yawPrev[-1]

    delta = steerPrev[-1]

    maxSteer = np.max(data[:, 3::dim])
    minSteer = np.min(data[:, 3::dim])
    steerAngles = np.linspace(minSteer, maxSteer, num=200)

    if animate:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8), gridspec_kw={'width_ratios': [3, 2]})
        camera = Camera(fig)

    for i in range(nDrawnSamples):
        # ---------
        currentState = np.array([truckState.x, truckState.y, truckState.yaw, delta])

        c_sample_seed[:dim] = currentState
        c_sample = torch.from_numpy(np.repeat([c_sample_seed],num_viz,axis=0)).float().to(DEVICE)

        y_viz = network(torch.randn(num_viz, z_dim).to(DEVICE), c_sample, encode=False).cpu().detach().numpy() # y_viz is sample in state space, zviz in latent space (3D)

        kdes = np.array([gaussian_kde(y_viz[:, i * dim - 1]) for i in range(1, nDrawnSamples + 1)])
        # ---------

        delta = steerAngles[np.argmax(kdes[0].pdf(steerAngles))]

        truckState.x += v * np.cos(truckState.yaw) * dt
        truckState.y += v * np.sin(truckState.yaw) * dt
        truckState.yaw += v / L * np.tan(delta) * dt

        ux = int(np.floor(truckState.x * gridSize)) # right pointing image axis
        vy = int(np.floor(gridSize * (1 - truckState.y))) # down pointing image axis
        if ux >= 0 and ux < gridSize and vy >= 0 and vy < gridSize:
            if not occGrid[vy,ux] > 0: # if collision
                collision = True
                break
            collision = False
        else:
            collision = True
            break

        if animate:
            ax1.scatter(y_viz[:,::dim] * gridSize, y_viz[:,1::dim] * gridSize, color='green', s=70, alpha=0.006, label='CVAE predicted (x,y) states') # nn samples
            ax1.quiver(goalStateCondition[vizIdx, 0] * gridSize, goalStateCondition[vizIdx, 1] * gridSize, np.cos(goalStateCondition[vizIdx, 2]), np.sin(goalStateCondition[vizIdx, 2]), color="blue", scale=11.0, width=0.012, label='Goal pose') # goal
            #ax1.quiver(truckState.x * gridSize, truckState.y * gridSize, np.cos(truckState.yaw), np.sin(truckState.yaw), color="red", scale=8.0, width=0.015) # init
            ax1.scatter(truckState.x * gridSize, truckState.y * gridSize, color="red") # init
            ax1.imshow(occGrid, extent=[0, gridSize, 0, gridSize], cmap='gray')
            ax1.set_xlabel('x', fontsize=12)
            ax1.set_ylabel('y', fontsize=12)

            ax2.plot(steerAngles, kdes[0].pdf(steerAngles), color='C0')
            ax2.set_xlabel('Steering angle', fontsize=12)
            ax2.set_ylabel('Sample density', fontsize=12)
            plt.show()

            camera.snap()

    CVAEendTime = time.time()
    
    if not collision:
        nCVAEsuccess += 1

        pdef.clear()
        nExploredStates = 0
        RRTstartTime = time.time()
        plan(pdef, initState=[xPrev[-1], yPrev[-1], yawPrev[-1]], goalState=[truckState.x, truckState.y, truckState.yaw])
        if pdef.getProblemDefinition().hasExactSolution():
            RRTendTime = time.time()
            RRTtime[nCompletedTests] = RRTendTime - RRTstartTime
            nRRTstates[nCompletedTests] = nExploredStates

            CVAEtime[nCompletedTests] = CVAEendTime - CVAEstartTime
            nCVAEstates[nCompletedTests] = nDrawnSamples # constant

            nRRTsuccess += 1
            nCompletedTests += 1
        else:
            nRRTfails += 1

        if animate:
            fps = 15
            animation = camera.animate(interval=1000/fps)
            print("saving animation")
            animation.save('animation.mp4')

            
    else:    
        nCVAEfails += 1

Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 1882 states
Info:    Solution found in 0.164063 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 1123 states
Info:    Solution found in 0.120802 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 2150 states
Info:    Solution found in 0.166493 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 433 states
Info:    Solution found in 0.027751 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 1020 states
Info:    Solution found in 0.069959 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 5362 states
Info:    Solution found in 0.451999 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 759 states
Inf

         at line 258 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/base/src/Planner.cpp
Error:   RRT: There are no valid initial states!
         at line 109 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/control/planners/rrt/src/RRT.cpp


Debug:   RRT: Discarded start state Compound state [
RealVectorState [0.839875 0.296967]
SO2State [3.16709]
]

Info:    No solution found after 0.000052 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 496 states
Info:    Solution found in 0.031660 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 1039 states
Info:    Solution found in 0.070836 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 544 states
Info:    Solution found in 0.046581 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 788 states
Info:    Solution found in 0.056521 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 558 states
Info:    Solution found in 0.052450 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 1618 

         at line 258 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/base/src/Planner.cpp
Error:   RRT: There are no valid initial states!
         at line 109 in /home/oscar_palfelt/MSc_thesis/ompl/src/ompl/control/planners/rrt/src/RRT.cpp


Debug:   RRT: Discarded start state Compound state [
RealVectorState [0.839875 0.296967]
SO2State [3.16709]
]

Info:    No solution found after 0.000044 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 301 states
Info:    Solution found in 0.022368 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 373 states
Info:    Solution found in 0.022943 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 839 states
Info:    Solution found in 0.063316 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 1821 states
Info:    Solution found in 0.164442 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 217 states
Info:    Solution found in 0.017716 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 2409 

KeyboardInterrupt: 

In [14]:
print('Mean CVAE CPU time: {}, std: {}'.format(np.mean(CVAEtime[:nCompletedTests]), np.std(CVAEtime[:nCompletedTests])))
print('Mean RRT CPU time: {}, std: {}'.format(np.mean(RRTtime[:nCompletedTests]), np.std(RRTtime[:nCompletedTests])))

print('Mean CVAE #states: {}, std: {}'.format(np.mean(nCVAEstates[:nCompletedTests]), np.std(nCVAEstates[:nCompletedTests])))
print('Mean RRT #states: {}, std: {}'.format(np.mean(nRRTstates[:nCompletedTests]), np.std(nRRTstates[:nCompletedTests])))

print('CVAE success rate: {}%'.format(nCVAEsuccess / (nCVAEsuccess + nCVAEfails) * 100))
print('RRT success rate: {}%'.format(nRRTsuccess / (nRRTsuccess + nRRTfails) * 100))

Mean CVAE CPU time: 0.8603104939827553, std: 0.20066124802941318
Mean RRT CPU time: 0.2252368871982281, std: 0.3255947780963888
Mean CVAE #states: 26.0, std: 0.0
Mean RRT #states: 13574.530769230769, std: 15644.557506434312
CVAE success rate: 24.813432835820894%
RRT success rate: 98.48484848484848%
